# ETL del dataset homicidios
Importación de librerías 

In [1]:
import pandas as pd
import numpy as np

Carga del dataset

In [5]:
df = pd.read_excel('../Data/homicidios.xlsx', sheet_name = 'HECHOS', na_values=['SD','sd'])

Visualización de las primeras 5 filas

In [6]:
df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Información de las columnas del dataset

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Conteo de nulos

In [8]:
df.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    9
ACUSADO                   23
dtype: int64

Extraigo varias muestras de forma aleatoria para verificar otro tipo de nulos

In [10]:
df.sample(10)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
576,2020-0056,1,2020-11-21,2020,11,21,13:00:00,13.0,"PAZ, GRAL. AV. Y DOCTOR RAMON CARRILLO",GRAL PAZ,...,NaN,"ROCA, CNEL. AV.","PAZ, GRAL. AV. y ROCA, CNEL. AV.",8,Point (99327.01838497 92326.91130631),-58.47064556,-34.69843756,MOTO-CARGAS,MOTO,CARGAS
222,2017-0098,1,2017-08-08,2017,8,8,12:10:00,12.0,JOSE BONIFACIO 2114,CALLE,...,2114.0,NaN,"BONIFACIO, JOSE 2114",7,Point (100563.22594607 99771.56223988),-58.45715752,-34.63132912,PEATON-AUTO,PEATON,AUTO
404,2018-0130,1,2018-12-03,2018,12,3,03:00:00,3.0,Gaona Av. y Segurola Av.,AVENIDA,...,NaN,SEGUROLA AV.,GAONA AV. y SEGUROLA AV.,10,Point (97531.36286491 100173.61049110),-58.49022072,-34.62770192,MOTO-PASAJEROS,MOTO,PASAJEROS
485,2019-0069,1,2019-08-17,2019,8,17,05:25:00,5.0,"GARAY, JUAN DE AV. 2874",AVENIDA,...,2874.0,NaN,"GARAY, JUAN DE AV. 2874",4,Point (105355.77079994 99994.97457130),-58.40489318,-34.62930162,MOTO-AUTO,MOTO,AUTO
540,2020-0020,1,2020-03-13,2020,3,13,17:30:00,17.0,"PUEYRREDON AV. Y GOMEZ, VALENTIN",AVENIDA,...,NaN,"GOMEZ, VALENTIN","PUEYRREDON AV. y GOMEZ, VALENTIN",3,Point (105281.81064188 102627.51469425),-58.40571596,-34.60557123,PEATON-MOTO,PEATON,MOTO
30,2016-0043,1,2016-03-30,2016,3,30,18:15:00,18.0,DELLEPIANE LUIS TTE GRAL COLECTORA Y MURGUIONDO,AVENIDA,...,NaN,MURGUIONDO,"DELLEPIANE, LUIS, TTE. GRAL. y MURGUIONDO",8,Point (98623.04338909 94933.06478837),-58.47832463,-34.67494416,PEATON-CARGAS,PEATON,CARGAS
477,2019-0061,1,2019-07-17,2019,7,17,18:40:00,18.0,11 DE SEPTIEMBRE DE 1888 Y MENDOZA,CALLE,...,NaN,MENDOZA,11 DE SEPTIEMBRE DE 1888 y MENDOZA,13,Point (101036.85652575 107926.87651681),-58.45200171,-34.55781320,PEATON-PASAJEROS,PEATON,PASAJEROS
262,2017-0148,1,2017-11-29,2017,11,29,10:45:00,10.0,AV. SAN JUAN Y LA RIOJA,AVENIDA,...,NaN,LA RIOJA,SAN JUAN AV. y LA RIOJA,3,Point (105155.42991407 100540.06938064),-58.40708125,-34.62438894,PEATON-PASAJEROS,PEATON,PASAJEROS
397,2018-0123,1,2018-11-12,2018,11,12,03:32:00,3.0,"Paz, Gral. Av. y Peron, Eva Av.",GRAL PAZ,...,NaN,"PERON, EVA AV.","PAZ, GRAL. AV. y PERON, EVA AV.",9,Point (96405.76679348 94981.38013791),-58.50251790,-34.67450313,AUTO-SD,AUTO,NaN
183,2017-0053,1,2017-05-05,2017,5,5,20:16:22,20.0,VIEYTES Y VILLARINO,CALLE,...,NaN,VILLARINO,VIEYTES y VILLARINO,4,Point (108050.74089604 97335.15201103),-58.37547846,-34.65326075,MOTO-CARGAS,MOTO,CARGAS


Elimino columnas inecesarias, luego de analizar las columnas se determino que las columnas *AAAA,MM,DD*

In [11]:
df.head(1)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [ ]:
echo "# Proyecto_Integrador_2" >> README.md
git init
git add .
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/Gerardgfc/Proyecto_Integrador_2.git
git push -u origin main